In [1]:
# Import packages
## tweepy - api to get data from twitter
import tweepy as tw
## hiding API keys - ref: https://youtu.be/CJjSOzb0IYs
import apikeys
## packages to data manipulation
import pandas as pd

In [2]:
# your Twitter API key and API secret
my_api_key = apikeys.key
my_api_secret = apikeys.secret
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
# Defining the query
search_query = '@LulaOficial'

In [4]:
# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,
              lang="pt",
              since="2022-09-15").items(50)

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,Guilherme Lima,World,Um social-democrata caminhando para esquerda. ...,False,2022-09-16 11:03:38+00:00,Foi o governo @LulaOficial. O governo que most...,None,Twitter Web App
1,Dayana🌈LulaNoPrimeiroTurno❤️ Conta Reserva❌,,,False,2022-09-16 11:03:37+00:00,SEXTOUUU🌻💕\nCopie e cole. Quem for votar no @...,None,Twitter for Android
2,traplev,"Recife, Brasil","((o twitter como recreio do play,\nao vivo ou ...",False,2022-09-16 11:03:37+00:00,RT @LulaOficial: Quem tem fome tem que comer. ...,None,Twitter for Android
3,Netinho santos,,,False,2022-09-16 11:03:37+00:00,@LulaOficial O seu PT vive dizendo que acabou ...,None,Twitter for Android
4,Ruy,"Belo Horizonte, Brasil","Petista, Administrador, eleitor de #LulaPresid...",False,2022-09-16 11:03:37+00:00,RT @LulaOficial: Bom dia! https://t.co/AvgYAF2A7W,None,Twitter for Android
